In [1]:
import h2o

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,5 hours 18 mins
H2O cluster timezone:,America/Santiago
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.2
H2O cluster version age:,6 months and 29 days !!!
H2O cluster name:,H2O_from_python_maravenag_szc3fv
H2O cluster total nodes:,1
H2O cluster free memory:,2.146 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [3]:
train = h2o.upload_file("data/train.csv", destination_frame="train")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
train.describe()
#test no tiene la clases :o

Rows:891
Cols:12




,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
type,int,int,int,string,enum,real,int,int,int,real,enum,enum
mins,1.0,0.0,1.0,NaN,,0.42,0.0,0.0,693.0,0.0,,
mean,446.0,0.3838383838383838,2.3086419753086447,NaN,,29.69911764705884,0.5230078563411893,0.3815937149270483,260318.5491679275,32.20420796857465,,
maxs,891.0,1.0,3.0,NaN,,80.0,8.0,6.0,3101298.0,512.3292,,
sigma,257.3538420152301,0.4865924542648575,0.8360712409770491,NaN,,14.526497332334035,1.1027434322934315,0.8060572211299488,471609.26868834975,49.69342859718089,,
zeros,0,549,0,0,,0,608,678,0,15,,
missing,0,0,0,0,0,177,0,0,230,0,687,2
0,1.0,0.0,3.0,"Braund, Mr. Owen Harris",male,22.0,1.0,0.0,nan,7.25,,S
1,2.0,1.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1.0,0.0,nan,71.2833,C85,C
2,3.0,1.0,3.0,"Heikkinen, Miss. Laina",female,26.0,0.0,0.0,nan,7.925,,S


In [5]:
train = train.drop("PassengerId")
train = train.drop("Name")
train = train.drop("Cabin")
train = train.drop("Ticket")

In [6]:
train, test = train.split_frame(ratios=[0.7])

In [7]:
train.head()

Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22,1,0,7.25,S
1,1,female,38,1,0,71.2833,C
1,3,female,26,0,0,7.925,S
1,1,female,35,1,0,53.1,S
0,3,male,35,0,0,8.05,S
0,3,male,nan,0,0,8.4583,Q
0,1,male,54,0,0,51.8625,S
0,3,male,2,3,1,21.075,S
1,3,female,4,1,1,16.7,S
1,1,female,58,0,0,26.55,S


In [8]:
train['Survived'] = train['Survived'].asfactor()
train['Pclass'] = train['Pclass'].asfactor()
train['Sex'] = train['Sex'].asfactor()
train['Embarked'] = train['Embarked'].asfactor()

test['Survived'] = test['Survived'].asfactor()
test['Pclass'] = test['Pclass'].asfactor()
test['Sex'] = test['Sex'].asfactor()
test['Embarked'] = test['Embarked'].asfactor()

#### Correr un autoML como baseline

In [9]:
from h2o.automl import H2OAutoML

In [10]:
target = "Survived"

In [11]:
train_cols = [x for x in train.col_names if x not in ['Survived']]

In [12]:
aml = H2OAutoML(max_runtime_secs = 100)
aml.train(x = train_cols, y = target,
          training_frame = train, validation_frame=test)

AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [14]:
lb = aml.leaderboard
print(lb)

model_id,auc,logloss
StackedEnsemble_BestOfFamily_0_AutoML_20181004_222555,0.87314,0.404598
StackedEnsemble_AllModels_0_AutoML_20181004_222555,0.871352,0.405517
GBM_grid_0_AutoML_20181004_222555_model_0,0.867584,0.411948
GBM_grid_0_AutoML_20181004_222555_model_14,0.86505,0.417263
GBM_grid_0_AutoML_20181004_222555_model_3,0.865017,0.423372
GBM_grid_0_AutoML_20181004_222555_model_2,0.864022,0.42445
GBM_grid_0_AutoML_20181004_222555_model_13,0.863022,0.450916
GBM_grid_0_AutoML_20181004_222555_model_1,0.861924,0.422671
GLM_grid_0_AutoML_20181004_222555_model_0,0.857987,0.437717
XRT_0_AutoML_20181004_222555,0.85703,0.507404


#### Ocupando un Autoencoder

In [15]:
from h2o.estimators import H2OAutoEncoderEstimator

In [16]:
autoencoder = H2OAutoEncoderEstimator(activation="Tanh",
                              hidden=[5],
                              ignore_const_cols=False,
                              epochs=1000)

In [17]:
autoencoder.train(x=train_cols, training_frame=train)

deeplearning Model Build progress: |██████████████████████████████████████| 100%


In [18]:
autoencoder._model_json["output"]["training_metrics"]


ModelMetricsAutoEncoder: deeplearning
** Reported on train data. **

MSE: 0.0050799698002500955
RMSE: 0.07127390686815263


In [40]:
train_deep_feats = autoencoder.deepfeatures(train, 0)
train_deep_feats = train_deep_feats.cbind(train['Survived'])
deep_cols = [x for x in train_deep_feats.col_names if x not in ['Survived']]

deepfeatures progress: |██████████████████████████████████████████████████| 100%


In [41]:
autoencoder.train(x=train_cols, training_frame=test)
autoencoder._model_json["output"]["training_metrics"]

deeplearning Model Build progress: |██████████████████████████████████████| 100%

ModelMetricsAutoEncoder: deeplearning
** Reported on train data. **

MSE: 0.005736324624306365
RMSE: 0.0757385280046184


In [42]:
test_deep_feats = autoencoder.deepfeatures(test, 0)
test_deep_feats = test_deep_feats.cbind(test['Survived'])

deepfeatures progress: |██████████████████████████████████████████████████| 100%


In [43]:
train_deep_feats.head()

DF.L1.C1,DF.L1.C2,DF.L1.C3,DF.L1.C4,DF.L1.C5,Survived
-0.680757,0.869347,0.808257,0.88243,-0.894286,0
0.863555,-0.895482,0.81057,-0.93484,0.95416,1
-0.681467,0.869444,0.806785,0.882418,0.942474,1
0.836916,0.86945,0.804929,-0.887755,0.939821,1
-0.678153,0.869552,0.807813,0.880667,-0.89452,0
-0.943842,-0.292784,-0.573188,0.429261,-0.385028,0
0.839938,0.86965,0.80551,-0.89062,-0.899347,0
-0.684709,0.869673,0.809938,0.884541,-0.891969,0
-0.684678,0.869397,0.808104,0.884527,0.943478,1
0.837805,0.87013,0.803775,-0.887199,0.939576,1


In [45]:
test_deep_feats.head()

DF.L1.C1,DF.L1.C2,DF.L1.C3,DF.L1.C4,DF.L1.C5,Survived
0.90759,-0.895927,-0.979157,-0.820317,0.844263,1
0.858789,0.859697,0.885163,-0.805421,-0.530245,1
-0.925509,0.739279,-0.97924,0.41451,0.845251,1
-0.950977,0.859489,0.88413,0.451284,0.826719,0
0.289047,-0.982072,0.167303,0.169471,-0.116128,1
-0.926462,0.741579,-0.979271,-0.819613,-0.48874,0
-0.950051,0.859744,0.884862,0.452669,0.829818,0
0.905766,-0.897567,-0.979227,-0.820238,0.844668,0
0.858312,-0.806626,0.884813,-0.805587,0.826089,1
0.863734,0.859999,0.88574,-0.806019,-0.528042,1


In [46]:
deep_feats_name = [x for x in train_deep_feats.col_names if x not in ["Survived"]]

In [48]:
aml2 = H2OAutoML(max_runtime_secs = 100)
aml2.train(x = deep_feats_name, y = target,
          training_frame = train_deep_feats, validation_frame=test_deep_feats)

AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [49]:
lb = aml2.leaderboard
print(lb)

model_id,auc,logloss
GBM_grid_0_AutoML_20181004_223107_model_8,0.861728,0.440265
GBM_grid_0_AutoML_20181004_223107_model_1,0.860211,0.431827
StackedEnsemble_BestOfFamily_0_AutoML_20181004_223107,0.859553,0.429481
GBM_grid_0_AutoML_20181004_223107_model_3,0.859336,0.435661
XRT_0_AutoML_20181004_223107,0.858042,0.546862
GBM_grid_0_AutoML_20181004_223107_model_2,0.857182,0.437872
StackedEnsemble_AllModels_0_AutoML_20181004_223107,0.857052,0.428944
DeepLearning_0_AutoML_20181004_223107,0.851191,0.440372
GBM_grid_0_AutoML_20181004_223107_model_4,0.850489,0.44762
GLM_grid_0_AutoML_20181004_223107_model_0,0.845558,0.450853


In [68]:
from h2o.estimators import H2ODeepLearningEstimator

In [69]:
model_deep = H2ODeepLearningEstimator(activation="Tanh", hidden=[5,3], epochs=1000, model_id="deep")

In [70]:
model_deep.train(x=deep_feats_name, y=target, training_frame=train_deep_feats, validation_frame=test_deep_feats)

deeplearning Model Build progress: |██████████████████████████████████████| 100%


In [71]:
model_deep.summary

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  deep


ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.13439675678288449
RMSE: 0.36660163226980386
LogLoss: 0.43131600960095834
Mean Per-Class Error: 0.20794910830795998
AUC: 0.8520987385819921
Gini: 0.7041974771639843
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3681643473811702: 


,0,1,Error,Rate
0,357.0,23.0,0.0605,(23.0/380.0)
1,86.0,156.0,0.3554,(86.0/242.0)
Total,443.0,179.0,0.1752,(109.0/622.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.3681643,0.7410926,143.0
max f2,0.0756810,0.7909605,358.0
max f0point5,0.6326101,0.8258427,126.0
max accuracy,0.3681643,0.8247588,143.0
max precision,0.9370949,1.0,0.0
max recall,0.0755630,1.0,396.0
max specificity,0.9370949,1.0,0.0
max absolute_mcc,0.6326101,0.6309343,126.0
max min_per_class_accuracy,0.3352086,0.7789474,230.0
max mean_per_class_accuracy,0.3681643,0.7920509,143.0


Gains/Lift Table: Avg response rate: 38.91 %



...


Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
,2018-10-04 22:45:52,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
,2018-10-04 22:45:52,0.022 sec,691111 obs/sec,10.0,1,6220.0,0.3666016,0.4313160,0.8520987,2.5702479,0.1752412,0.5713128,0.8987977,0.5818639,0.0,0.4869888
,2018-10-04 22:45:52,0.807 sec,792356 obs/sec,1000.0,100,622000.0,0.3652097,0.4235788,0.8614071,2.5702479,0.2106109,0.6394700,1.4099840,0.5652367,0.0,0.4684015
,2018-10-04 22:45:52,0.814 sec,791348 obs/sec,1000.0,100,622000.0,0.3666016,0.4313160,0.8520987,2.5702479,0.1752412,0.5713128,0.8987977,0.5818639,0.0,0.4869888


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
DF.L1.C3,1.0,1.0,0.3246925
DF.L1.C5,0.7228473,0.7228473,0.2347031
DF.L1.C2,0.4832063,0.4832063,0.1568935
DF.L1.C4,0.4613491,0.4613491,0.1497966
DF.L1.C1,0.4124342,0.4124342,0.1339143


<bound method ModelBase.summary of >